In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString
from shapely.ops import linemerge, unary_union

def create_upstream_charge_area_precise(edge, node, charge_station, distance=1000):
    """
    创建充电站精确上游1000m的道路面，确保边界精确。

    参数：
    - edge (GeoDataFrame): 包含道路边信息的地理空间数据框。
    - node (GeoDataFrame): 包含节点信息的地理空间数据框。
    - charge_station (DataFrame): 包含充电站信息的数据框。
    - distance (int): 上游的精确距离，例如1000米。

    返回：
    - charge_station_areas (GeoDataFrame): 包含每个充电站的上游道路面的地理空间数据框。
    """
    G = nx.DiGraph()
    G.add_nodes_from(node['id'])
    G.add_weighted_edges_from(edge[['u', 'v', 'length']].apply(tuple, axis=1))
    
    charge_station_areas = gpd.GeoDataFrame(columns=['charge_station_id', 'geometry'], crs='EPSG:4326')

    for index, station in charge_station.iterrows():
        station_id = station['id']
        node_id = station['node_id']
        path_edges = []
        visited = set()
        queue = [(node_id, 0, None)]  # (current_node, accumulated_distance, last_edge)

        while queue:
            current_node, acc_dist, last_edge = queue.pop(0)
            if current_node not in visited:
                visited.add(current_node)
                for u, v, length in G.in_edges(current_node, data='length'):
                    if acc_dist + length < distance:
                        queue.append((u, acc_dist + length, (u, v, length)))
                    elif acc_dist + length == distance:
                        path_edges.append((u, v, length))
                        break
                    else:
                        # 截断超出部分
                        if last_edge:
                            u, v, len_original = last_edge
                            proportion = (distance - acc_dist) / length
                            edge_geom = edge[(edge['u'] == u) & (edge['v'] == v)]['geometry'].iloc[0]
                            cut_point = edge_geom.interpolate(proportion * edge_geom.length)
                            new_geom = LineString([cut_point, edge_geom.coords[-1]])
                            path_edges.append((u, v, distance - acc_dist, new_geom))
                        break

        if path_edges:
            # 处理路径边生成面
            df_edges = pd.DataFrame(path_edges, columns=['u', 'v', 'length', 'geometry'])
            area = generate_plane(df_edges, dist=100)
            if not area.empty:
                total_area = unary_union(area['geometry'])
                charge_station_areas = charge_station_areas.append({'charge_station_id': station_id, 'geometry': total_area}, ignore_index=True)

    return charge_station_areas


In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Polygon

def create_charge_station_upstream_area(edge, node, charge_station, radius=1000):
    """
    为每个充电站创建上游1000m的道路面

    参数：
    - edge (GeoDataFrame): 包含道路边信息的地理空间数据框。
    - node (GeoDataFrame): 包含节点信息的地理空间数据框。
    - charge_station (DataFrame): 包含充电站信息的数据框。
    - radius (int): 上游区域的半径（默认1000米）。

    返回：
    - charge_station_areas (GeoDataFrame): 包含每个充电站的上游道路面的地理空间数据框。
    """

    # 初始化一个空的GeoDataFrame用于存储结果
    charge_station_areas = gpd.GeoDataFrame(columns=['charge_station_id', 'geometry'], crs='EPSG:4326')

    # 构建网络
    G = nx.DiGraph()
    G.add_nodes_from(node['id'])
    G.add_weighted_edges_from(edge[['u', 'v', 'length']].apply(tuple, axis=1))

    # 遍历充电站
    for index, row in charge_station.iterrows():
        station_id = row['id']
        node_id = row['node_id']
        
        # 使用NetworkX查找所有在1000m半径内的节点
        reachable_nodes = nx.single_source_dijkstra_path_length(G, node_id, cutoff=radius, weight='length')
        
        # 从edge DataFrame中提取这些节点对应的边
        upstream_edges = edge[edge['u'].isin(reachable_nodes.keys()) | edge['v'].isin(reachable_nodes.keys())]

        # 生成道路面
        upstream_area = generate_plane(upstream_edges, dist=100)
        
        # 合并所有道路面
        if not upstream_area.empty:
            total_area = gpd.GeoSeries(upstream_area['geometry'].unary_union)
            charge_station_areas = charge_station_areas.append({'charge_station_id': station_id, 'geometry': total_area}, ignore_index=True)

    return charge_station_areas
